<a href="https://colab.research.google.com/github/jh941213/ChatGPT/blob/main/autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(94) # Seed 고정

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.1.0"
!pip install autogluon.tabular
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon  # autogluon==0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from autogluon.tabular import TabularPredictor, TabularDataset

# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/dacon3/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon3/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/dacon3/sample_submission.csv')


In [ ]:
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

# Quantify qualitative variables
# 정성적 변수는 LabelEncoder를 사용하여 숫자로 인코딩됩니다.
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i])
print('Done.')

Done.
Done.


In [ ]:
# Remove unlabeled data
# 훈련 세트에서 레이블이 지정되지 않은 데이터가 제거되고 숫자 레이블 열이 추가됩니다.
train = train.dropna()

column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')



Done.


In [ ]:
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

In [ ]:
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
5,TRAIN_000005,4,13,1545.0,1900.0,0,0,119,11618,4,93,11278,47,199.0,21,8,20452.0,3435,Not_Delayed,1
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,119,11618,28,47,10721,19,200.0,26,8,19393.0,3495,Not_Delayed,1
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,59,10821,4,74,11057,31,361.0,23,10,19393.0,4083,Not_Delayed,1
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,93,11278,47,277,14122,36,204.0,21,0,19393.0,241,Delayed,2
12,TRAIN_000012,1,12,1015.0,1145.0,0,0,72,11042,33,94,11292,5,1201.0,23,10,19393.0,5171,Not_Delayed,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999962,TRAIN_999962,10,11,600.0,2003.0,0,0,310,14683,42,256,13930,11,1041.0,22,8,20304.0,488,Not_Delayed,1
999963,TRAIN_999963,5,2,1759.0,1926.0,0,0,204,12953,30,93,11278,47,214.0,23,3,20452.0,5204,Delayed,2
999969,TRAIN_999969,10,10,940.0,1056.0,0,0,223,13256,42,169,12266,42,316.0,19,8,20378.0,5350,Delayed,2
999985,TRAIN_999985,8,8,1914.0,2039.0,0,0,296,14492,31,183,12451,7,407.0,14,4,20436.0,1499,Not_Delayed,1


In [ ]:
# 훈련 데이터와 검증 데이터를 나눕니다
#train_data = TabularDataset(train_df)
#predictor = TabularPredictor(label='Delay_num', eval_metric='log_loss').fit(train_data,presets='best_quality',num_stack_levels=3,verbosity=2)
predictor = TabularPredictor(label='Delay_num', eval_metric='log_loss')
predictor.fit(train, num_gpus=1, presets='best_quality', verbosity=2)


No path specified. Models will be saved in: "AutogluonModels/ag-20230407_111452/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230407_111452/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    255001
Train Data Columns: 19
Label Column: Delay_num
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 2]
	If 'binary' is not the correct problem_type, please manually specify the problem_type par

In [ ]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,-1.033899e-07,3.356291,88.738178,3.356291,88.738178,1,True,4
1,LightGBMLarge_BAG_L1,-1.033899e-07,34.929126,244.690564,34.929126,244.690564,1,True,13
2,LightGBMXT_BAG_L1,-1.033899e-07,4.665355,81.209692,4.665355,81.209692,1,True,3
3,NeuralNetFastAI_BAG_L1,-1.033899e-07,5.746506,794.607525,5.746506,794.607525,1,True,10
4,WeightedEnsemble_L2,-1.033899e-07,3.364797,101.643687,0.008506,12.905509,2,True,14
5,NeuralNetTorch_BAG_L1,-1.136712e-07,0.697320,1509.115059,0.697320,1509.115059,1,True,12
6,CatBoost_BAG_L1,-2.702107e-06,0.481501,145.480989,0.481501,145.480989,1,True,7
7,XGBoost_BAG_L1,-1.036332e-05,1.861631,28.184349,1.861631,28.184349,1,True,11
8,ExtraTreesEntr_BAG_L1,-1.375088e-03,4.875138,20.165198,4.875138,20.165198,1,True,9
9,ExtraTreesGini_BAG_L1,-1.512677e-03,4.480563,19.753566,4.480563,19.753566,1,True,8


In [ ]:
model_to_use = predictor.get_model_best()


In [ ]:
model_pred = predictor.predict(test_x, model=model_to_use)

KeyError: ignored

In [ ]:
test_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,1900.0,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,19393.0,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,California,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TEST_999995,6,2,600.0,2240.0,0,0,LAX,12892,California,DEN,11292,Colorado,862.0,Southwest Airlines Co.,WN,19393.0,N720WN
999996,TEST_999996,6,8,600.0,648.0,0,0,BUF,10792,New York,ORD,13930,Illinois,473.0,United Air Lines Inc.,UA,19977.0,N401UA
999997,TEST_999997,6,11,1710.0,1907.0,0,0,IAD,12264,California,DTW,11433,Michigan,383.0,Mesa Airlines Inc.,UA,20378.0,N510MJ
999998,TEST_999998,11,17,600.0,500.0,0,0,SAN,14679,California,BOS,10721,Massachusetts,2588.0,JetBlue Airways,B6,20409.0,N986JB


In [ ]:
test_x = test.drop(columns=['ID'])

In [ ]:
train.columns

Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'Delay',
       'Delay_num'],
      dtype='object')

In [ ]:
test.columns

Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number'],
      dtype='object')

In [ ]:
test_pred = predictor.predict_proba(test_x)

KeyError: ignored

In [ ]:
sample_submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)

In [ ]:
sample_submission.to_csv('/kaggle/working/submission2.csv', index=True)